### 국회 정당, 원내 대변인 보도자료 발표 및 브리핑, 논평 자료 전처리 파일

In [4]:
# import

import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

from gensim.models import Doc2Vec
import sentencepiece as spm

from tqdm import trange
import os

In [5]:
# 기타 설정


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)



C:\Users\shjoo\AppData\Local\Temp\ipykernel_13320\655236758.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [7]:
# data read

npp = pd.read_csv("NPP_final_0729.csv")
tmp = pd.read_csv("TMP_final_0729.csv")

In [8]:
# 결측치 확인 및 제거

npp.isnull().sum()



category    0
year        0
quarter     0
content     0
dtype: int64

In [9]:
tmp.isnull().sum()

title      0
year       0
quarter    0
content    2
dtype: int64

In [10]:
tmp = tmp.dropna()

In [18]:
# npp & tmp 합치기

npp['target'] = '국민의힘'
tmp['target'] = '민주당'

dataset_all = pd.concat([npp,tmp],ignore_index= True)
dataset_all.content[0]

'‘빗물을 나르는 고속도로’라 불리는 대심도 빗물터널 조금 전에 둘러보고 왔다. 신월 빗물 전용 배수시설은 집중호우 때마다 주변 지역이 물바다가 되는 현상을 막고자 2013년 5월에 시작해서 2020년 5월까지 만 7년의 긴 기간 공사를 거쳐서 완공된 국내 최초의 대심도 빗물 저류 시설로 알고 있다. 물은 환경의 영역이기도 하지만 그보다 더 우선돼야 할 것은 치수의 영역이라고 저는 생각한다. 국가적으로 물길을 다스리는 모든 행정을 일컬어서 치수라고 하는데 이 치수가 제대로 되지 않으면 국민의 생명과 안전, 그리고 국민의 소중한 재산을 심대하게 침해하기 때문에 어떤 경우에도 목숨보다 더 우선되는 것이 없다는 차원에서 보면 치수가 가장 중요한 일이라고 말씀드리고 싶다. 특히 가뭄과 홍수에도 제대로 대응하려면 물은 치수의 차원에서 먼저 다뤄야 할 자원이라고 말씀드리고 싶다. 지난 정권 초기였던 2018년에 문재인 정부 시절입니다만, 물관리 일원화 조치를 한다고 하면서 국토해양부, 환경부, 농림축산식품부에 나눠졌던 물관리 업무를 환경부로 일원화하는 조치가 있었다. 그 당시에도 환경부가 전국의 지류·지천·하수 등의 물관리 업무 전반을 담당할 역량과 경험이 되는지에 대한 많은 의문이 제기되었지만, 이번 폭우 사태를 겪으면서 그 많은 의문이 현실로 드러나고 있는 것이란 생각을 하지 않을 수 없다. 물관리의 실패는 국민의 생명과 안전을 직접적 위협하고, 심대한 재산적 피해를 야기하는 것이니만큼 이 점에 대한 문제점의 보완이 절실하다고 하겠다. 과거 2010년과 2011년 당시에 서울 도심에 큰 물난리가 났다. 그때 대심도 빗물터널이 필요하다고 그런 필요성이 제기되었고 그 당시 시정을 책임지었던 서울시장이 오세훈 시장이었다. 그래서 오세훈 시장은 2021년까지 총 8,500억원 들여서 서울 도심의 상습 침수지역 7곳에 대한 대심도 배수관 설치하겠다. 이렇게 결정한 바가 있었다. 7곳이 광화문, 신월, 용산, 사당역, 강남역, 동작, 강동 이렇게 7곳이었는데, 그렇게 건설하는 

In [19]:
# content만 남기기

dataset_all = dataset_all[['content','target']]
dataset_all.head()

,content,target
0,‘빗물을 나르는 고속도로’라 불리는 대심도 빗물터널 조금 전에 둘러보고 왔다. 신월...,국민의힘
1,"오늘 한 언론에 보도된 무소속 박완주 의원의 검찰 공소장 안에는, 강력 성범죄자의 ...",국민의힘
2,전국적인 집중호우로 많은 국민들이 희생되고 막대한 재산 피해가 발생했다. 당장 전면...,국민의힘
3,더불어민주당이 윤석열 정부의 ‘시행령 추진’에 본격적으로 제동을 걸고 있다. KBS...,국민의힘
4,"그칠 줄 모르던 비가 잠시 소강상태에 접어들었지만, 폭우가 할퀴고 간 자리에 남겨진...",국민의힘


In [20]:
# subtoken 단위로 학습을 시키기 위해 sentencepiece 를 먼저 학습 시킨다.

f = open("allsentence.txt","w")
f.write("".join(dataset_all).replace("\xa0", ""))
f.close()

In [22]:
#subword 단위
corpus = "allsentence.txt"
prefix = "sentence"
vocab_size = 40 #최대가 왜 47인지 모르겠음.
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰